# Comparing optimizers in a sign language prediction application
Deep Learning (20/21) `WMAI017-05.2020-2021.2A` - by Jeroen Overschie and Loran Knol.

Uses [data](https://www.kaggle.com/grassknoted/asl-alphabet) describing the ASL alphabet and tries to classify the images correctly using an adapted custom Neural Network, built with TensorFlow/Keras. Runs the model fitting multiple times for various optimizers, such that we can compare various optimizers against each other.

## Loading the dataset

In [5]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

1.15.5
2.1.5


In [6]:
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

Some various parameters.

In [7]:
data_dir = "../datasets/asl_alphabet_train/asl_alphabet_train"
target_size = (64, 64)
target_dims = (64, 64, 3)
n_classes = 29
val_frac = 0.1
batch_size = 64

Random seed

In [8]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

Use ImageDataGenerator

In [9]:
data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_frac)
train_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(data_dir, target_size=target_size, batch_size=batch_size, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Define the model
Define custom Neural Network, see [link](https://www.kaggle.com/dansbecker/running-kaggle-kernels-with-a-gpu). Convolutes the images, activates using ReLU and performes Dropout.

In [10]:
my_model = Sequential()
my_model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=target_dims))
my_model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
my_model.add(Dropout(0.5))
my_model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
my_model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
my_model.add(Flatten())
my_model.add(Dropout(0.5))
my_model.add(Dense(512, activation='relu'))
my_model.add(Dense(n_classes, activation='softmax'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Additionally, define a custom _Callback_ that also saves the time spent per epoch. Inherits from `CSVLogger` so also saves the loss per epoch to a CSV file.

In [11]:
import time
from keras.callbacks import CSVLogger

class EpochLoss(CSVLogger):
    def on_train_begin(self, logs={}):
        self.times = []
        super(EpochLoss, self).on_train_begin()

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()
        super(EpochLoss, self).on_epoch_begin(epoch, logs)

    def on_epoch_end(self, epoch, logs={}):
        logs['epoch_time'] = time.time() - self.epoch_time_start
        super(EpochLoss, self).on_epoch_end(epoch, logs)

## Optimizers
Configure a line-up of optimizers to use in the benchmark.

Yogi

In [12]:
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.util.tf_export import tf_export

# if you use tf.keras:
from keras import backend as K  
from keras.optimizers import Optimizer

# if you use standalone Keras
# from keras import backend as K
# from keras.optimizers import Optimizer
    
class Yogi(Optimizer):
    """Yogi optimizer.
    Default parameters follow those provided in the original paper.
    Arguments:
      lr: float >= 0. Learning rate.
      beta_1: float, 0 < beta < 1. Generally close to 1.
      beta_2: float, 0 < beta < 1. Generally close to 1.
      epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
      decay: float >= 0. Learning rate decay over each update.
      amsgrad: boolean. Whether to apply the AMSGrad variant of this
          algorithm from the paper "On the Convergence of Adam and
          Beyond".
    """

    def __init__(self,
               lr=0.001,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=None,
               decay=0.00000001,
               amsgrad=False,
               **kwargs):
        super(Yogi, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [state_ops.assign_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (  # pylint: disable=g-no-augmented-assignment
                1. / (1. + self.decay * math_ops.cast(self.iterations,
                                                    K.dtype(self.decay))))

        t = math_ops.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (
            K.sqrt(1. - math_ops.pow(self.beta_2, t)) /
            (1. - math_ops.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            #v_t = (self.beta_2 * v) + (1. - self.beta_2) * math_ops.square(g) # from amsgrad
            v_t = v - (1-self.beta_2)*K.sign(v-math_ops.square(g))*math_ops.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(state_ops.assign(m, m_t))
            self.updates.append(state_ops.assign(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(state_ops.assign(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'lr': float(K.get_value(self.lr)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad
            }
        base_config = super(Yogi, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

RAdam

In [13]:
import tensorflow as tf
from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import resource_variable_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.training import optimizer

class RAdamOptimizer(optimizer.Optimizer):

    """
    RAdam optimizer : On The Variance Of The Adaptive Learning Rate And Beyond
    https://arxiv.org/abs/1908.03265
    """

    def __init__(self,
                 learning_rate=0.001,
                 beta1=0.9,
                 beta2=0.999,
                 epsilon=1e-8,
                 weight_decay=0.,
                 use_locking=False,
                 name="RAdam"):

        super(RAdamOptimizer, self).__init__(use_locking, name)
        self._lr = learning_rate
        self._beta1 = beta1
        self._beta2 = beta2
        self._epsilon = epsilon
        self._weight_decay = weight_decay

        self._lr_t = None
        self._step_t = None
        self._beta1_t = None
        self._beta2_t = None
        self._epsilon_t = None
        self._weight_decay_t = None

    def _get_beta_accumulators(self):
        with ops.init_scope():
            if context.executing_eagerly():
                graph = None
            else:
                graph = ops.get_default_graph()
            return (self._get_non_slot_variable("step", graph=graph),
                    self._get_non_slot_variable("beta1_power", graph=graph),
                    self._get_non_slot_variable("beta2_power", graph=graph))

    def _create_slots(self, var_list):
        first_var = min(var_list, key=lambda x: x.name)
        self._create_non_slot_variable(initial_value=1.0, name="step", colocate_with=first_var)
        self._create_non_slot_variable(initial_value=self._beta1, name="beta1_power", colocate_with=first_var)
        self._create_non_slot_variable(initial_value=self._beta2, name="beta2_power", colocate_with=first_var)

        for v in var_list:
            self._zeros_slot(v, "m", self._name)
            self._zeros_slot(v, "v", self._name)

    def _prepare(self):
        lr = self._lr
        beta1 = self._beta1
        beta2 = self._beta2
        epsilon = self._epsilon
        weight_decay = self._weight_decay

        self._lr_t = ops.convert_to_tensor(lr, name="learning_rate")
        self._beta1_t = ops.convert_to_tensor(beta1, name="beta1")
        self._beta2_t = ops.convert_to_tensor(beta2, name="beta2")
        self._epsilon_t = ops.convert_to_tensor(epsilon, name="epsilon")
        self._weight_decay_t = ops.convert_to_tensor(weight_decay, name="weight_decay")

    def _apply_dense(self, grad, var):
        return self._resource_apply_dense(grad, var)

    def _resource_apply_dense(self, grad, var):
        step, beta1_power, beta2_power = self._get_beta_accumulators()
        step = math_ops.cast(step, var.dtype.base_dtype)
        beta1_power = math_ops.cast(beta1_power, var.dtype.base_dtype)
        beta2_power = math_ops.cast(beta2_power, var.dtype.base_dtype)
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)

        beta1_t = math_ops.cast(self._beta1_t, var.dtype.base_dtype)
        beta2_t = math_ops.cast(self._beta2_t, var.dtype.base_dtype)
        epsilon_t = math_ops.cast(self._epsilon_t, var.dtype.base_dtype)

        sma_inf = 2.0 / (1.0 - beta2_t) - 1.0
        sma_t = sma_inf - 2.0 * step * beta2_power / (1.0 - beta2_power)

        m = self.get_slot(var, "m")
        m_t = state_ops.assign(m, beta1_t * m + (1.0 - beta1_t) * grad, use_locking=self._use_locking)
        mhat_t = m_t / (1.0 - beta1_power)

        v = self.get_slot(var, "v")
        v_t = state_ops.assign(v, beta2_t * v + (1.0 - beta2_t) * math_ops.square(grad), use_locking=self._use_locking)
        vhat_t = math_ops.sqrt(v_t / ((1.0 - beta2_power) + epsilon_t))

        r_t = math_ops.sqrt( ((sma_t - 4.0) * (sma_t - 2.0) * sma_inf) / ((sma_inf - 4.0) * (sma_inf - 2.0) * sma_t) )

        var_t = tf.cond(sma_t >= 5.0, lambda : r_t * mhat_t / (vhat_t + epsilon_t), lambda : mhat_t)

        if self._weight_decay > 0.0:
            var_t += math_ops.cast(self._weight_decay_t, var.dtype.base_dtype) * var

        var_update = state_ops.assign_sub(var, lr_t * var_t, use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]

        return control_flow_ops.group(*updates)

    def _apply_sparse_shared(self, grad, var, indices, scatter_add):
        step, beta1_power, beta2_power = self._get_beta_accumulators()
        step = math_ops.cast(step, var.dtype.base_dtype)
        beta1_power = math_ops.cast(beta1_power, var.dtype.base_dtype)
        beta2_power = math_ops.cast(beta2_power, var.dtype.base_dtype)
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)

        beta1_t = math_ops.cast(self._beta1_t, var.dtype.base_dtype)
        beta2_t = math_ops.cast(self._beta2_t, var.dtype.base_dtype)
        epsilon_t = math_ops.cast(self._epsilon_t, var.dtype.base_dtype)

        sma_inf = 2.0 / (1.0 - beta2_t) - 1.0
        sma_t = sma_inf - 2.0 * step * beta2_power / (1.0 - beta2_power)

        m = self.get_slot(var, "m")
        m_scaled_g_values = grad * (1 - beta1_t)
        m_t = state_ops.assign(m, m * beta1_t, use_locking=self._use_locking)

        with ops.control_dependencies([m_t]):
            m_t = scatter_add(m, indices, m_scaled_g_values)

        mhat_t = m_t / (1.0 - beta1_power)

        v = self.get_slot(var, "v")
        v_scaled_g_values = (grad * grad) * (1 - beta2_t)
        v_t = state_ops.assign(v, v * beta2_t, use_locking=self._use_locking)

        with ops.control_dependencies([v_t]):
            v_t = scatter_add(v, indices, v_scaled_g_values)

        vhat_t = math_ops.sqrt(v_t / (1.0 - beta2_power) + epsilon_t)

        r_t = math_ops.sqrt( ((sma_t - 4.0) * (sma_t - 2.0) * sma_inf) / ((sma_inf - 4.0) * (sma_inf - 2.0) * sma_t) )

        var_t = tf.cond(sma_t >= 5.0, lambda : r_t * mhat_t / (vhat_t + epsilon_t), lambda : mhat_t)

        if self._weight_decay > 0.0:
            var_t += math_ops.cast(self._weight_decay_t, var.dtype.base_dtype) * var

        var_update = state_ops.assign_sub(var, lr_t * var_t, use_locking=self._use_locking)

        updates = [var_update, m_t, v_t]

        return control_flow_ops.group(*updates)

    def _apply_sparse(self, grad, var):
        return self._apply_sparse_shared(
            grad.values,
            var,
            grad.indices,
            lambda x, i, v: state_ops.scatter_add(x, i, v, use_locking=self._use_locking))

    def _resource_scatter_add(self, x, i, v):
        with ops.control_dependencies([resource_variable_ops.resource_scatter_add(x.handle, i, v)]):
            return x.value()

    def _resource_apply_sparse(self, grad, var, indices):
        return self._apply_sparse_shared(grad, var, indices, self._resource_scatter_add)

    def _finish(self, update_ops, name_scope):
        with ops.control_dependencies(update_ops):
            step, beta1_power, beta2_power = self._get_beta_accumulators()
            with ops.colocate_with(beta1_power):
                update_step = step.assign(step + 1.0, use_locking=self._use_locking)
                update_beta1 = beta1_power.assign(beta1_power * self._beta1_t, use_locking=self._use_locking)
                update_beta2 = beta2_power.assign(beta2_power * self._beta2_t, use_locking=self._use_locking)
        return control_flow_ops.group(*update_ops + [update_step, update_beta1, update_beta2], name=name_scope)

AdaBelief

In [14]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""AdaBelief for TensorFlow.
Modified from tensorflow/tensorflow/python/training/adam.py
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import resource_variable_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.training import optimizer
from tensorflow.python.training import training_ops
from tensorflow.python.util.tf_export import tf_export


@tf_export(v1=["train.AdaBeliefOptimizer"])
class AdaBeliefOptimizer(optimizer.Optimizer):
    """Optimizer that implements the Adam algorithm.
    References:
    Adam - A Method for Stochastic Optimization:
      [Kingma et al., 2015](https://arxiv.org/abs/1412.6980)
      ([pdf](https://arxiv.org/pdf/1412.6980.pdf))
    """

    def __init__(self,
               learning_rate=0.001,
               beta1=0.9,
               beta2=0.999,
               epsilon=1e-8,
               use_locking=False,
               name="AdaBelief", amsgrad = False):
        r"""Construct a new Adam optimizer.
        Initialization:
        $$m_0 := 0 \text{(Initialize initial 1st moment vector)}$$
        $$v_0 := 0 \text{(Initialize initial 2nd moment vector)}$$
        $$t := 0 \text{(Initialize timestep)}$$
        The update rule for `variable` with gradient `g` uses an optimization
        described at the end of section 2 of the paper:
        $$t := t + 1$$
        $$\text{lr}_t := \mathrm{learning_rate} *
          \sqrt{1 - \beta_2^t} / (1 - \beta_1^t)$$
        $$m_t := \beta_1 * m_{t-1} + (1 - \beta_1) * g$$
        $$v_t := \beta_2 * v_{t-1} + (1 - \beta_2) * g * g$$
        $$\text{variable} := \text{variable} -
          \text{lr}_t * m_t / (\sqrt{v_t} + \epsilon)$$
        The default value of 1e-8 for epsilon might not be a good default in
        general. For example, when training an Inception network on ImageNet a
        current good choice is 1.0 or 0.1. Note that since AdamOptimizer uses the
        formulation just before Section 2.1 of the Kingma and Ba paper rather than
        the formulation in Algorithm 1, the "epsilon" referred to here is "epsilon
        hat" in the paper.
        The sparse implementation of this algorithm (used when the gradient is an
        IndexedSlices object, typically because of `tf.gather` or an embedding
        lookup in the forward pass) does apply momentum to variable slices even if
        they were not used in the forward pass (meaning they have a gradient equal
        to zero). Momentum decay (beta1) is also applied to the entire momentum
        accumulator. This means that the sparse behavior is equivalent to the dense
        behavior (in contrast to some momentum implementations which ignore momentum
        unless a variable slice was actually used).
        Args:
          learning_rate: A Tensor or a floating point value.  The learning rate.
          beta1: A float value or a constant float tensor. The exponential decay
            rate for the 1st moment estimates.
          beta2: A float value or a constant float tensor. The exponential decay
            rate for the 2nd moment estimates.
          epsilon: A small constant for numerical stability. This epsilon is
            "epsilon hat" in the Kingma and Ba paper (in the formula just before
            Section 2.1), not the epsilon in Algorithm 1 of the paper.
          use_locking: If True use locks for update operations.
          name: Optional name for the operations created when applying gradients.
            Defaults to "Adam".
        @compatibility(eager)
        When eager execution is enabled, `learning_rate`, `beta1`, `beta2`, and
        `epsilon` can each be a callable that takes no arguments and returns the
        actual value to use. This can be useful for changing these values across
        different invocations of optimizer functions.
        @end_compatibility
        """
        super(AdaBeliefOptimizer, self).__init__(use_locking, name)
        self._lr = learning_rate
        self._beta1 = beta1
        self._beta2 = beta2
        self._epsilon = epsilon
        self.amsgrad = amsgrad

        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None
        self._beta1_t = None
        self._beta2_t = None
        self._epsilon_t = None

    def _get_beta_accumulators(self):
        with ops.init_scope():
          if context.executing_eagerly():
            graph = None
          else:
            graph = ops.get_default_graph()
          return (self._get_non_slot_variable("beta1_power", graph=graph),
                  self._get_non_slot_variable("beta2_power", graph=graph))

    def _create_slots(self, var_list):
        # Create the beta1 and beta2 accumulators on the same device as the first
        # variable. Sort the var_list to make sure this device is consistent across
        # workers (these need to go on the same PS, otherwise some updates are
        # silently ignored).
        first_var = min(var_list, key=lambda x: x.name)
        self._create_non_slot_variable(
            initial_value=self._beta1, name="beta1_power", colocate_with=first_var)
        self._create_non_slot_variable(
            initial_value=self._beta2, name="beta2_power", colocate_with=first_var)

        # Create slots for the first and second moments.
        for v in var_list:
          self._zeros_slot(v, "m", self._name)
          self._zeros_slot(v, "v", self._name)
          self._zeros_slot(v, "vhat", self._name)

    def _prepare(self):
        lr = self._lr
        beta1 = self._beta1
        beta2 = self._beta2
        epsilon = self._epsilon

        self._lr_t = ops.convert_to_tensor(lr, name="learning_rate")
        self._beta1_t = ops.convert_to_tensor(beta1, name="beta1")
        self._beta2_t = ops.convert_to_tensor(beta2, name="beta2")
        self._epsilon_t = ops.convert_to_tensor(epsilon, name="epsilon")

    def _apply_dense(self, grad, var):
          graph = None if context.executing_eagerly() else ops.get_default_graph()
          beta1_power = math_ops.cast(self._get_non_slot_variable("beta1_power", graph=graph), var.dtype.base_dtype)
          beta2_power = math_ops.cast(self._get_non_slot_variable("beta2_power", graph=graph), var.dtype.base_dtype)
          lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)

          beta1_t = math_ops.cast(self._beta1_t, var.dtype.base_dtype)
          beta2_t = math_ops.cast(self._beta2_t, var.dtype.base_dtype)
          epsilon_t = math_ops.cast(self._epsilon_t, var.dtype.base_dtype)

          step_size = (lr_t * math_ops.sqrt(1 - beta2_power) / (1 - beta1_power))

          # m_t = beta1 * m + (1 - beta1) * g_t
          m = self.get_slot(var, "m")
          m_scaled_g_values = grad * (1 - beta1_t)
          m_t = state_ops.assign(m, beta1_t * m + m_scaled_g_values, use_locking=self._use_locking)

          # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
          v = self.get_slot(var, "v")
          v_scaled_g_values = (grad -m_t) * (grad - m_t) * (1 - beta2_t)
          v_t = state_ops.assign(v, beta2_t * v + v_scaled_g_values + epsilon_t, use_locking=self._use_locking)

          # amsgrad
          vhat = self.get_slot(var, "vhat")
          if self.amsgrad:
              vhat_t = state_ops.assign(vhat, math_ops.maximum(v_t, vhat))
              v_sqrt = math_ops.sqrt(vhat_t)
          else:
              vhat_t = state_ops.assign(vhat, vhat)
              v_sqrt = math_ops.sqrt(v_t)

          # Compute the bounds
          step_size = step_size / (v_sqrt + epsilon_t)
          bounded_lr = m_t * step_size

          var_update = state_ops.assign_sub(var, bounded_lr, use_locking=self._use_locking)
          return control_flow_ops.group(*[var_update, m_t, v_t, vhat_t])

    def _resource_apply_dense(self, grad, var):
          graph = None if context.executing_eagerly() else ops.get_default_graph()
          beta1_power = math_ops.cast(self._get_non_slot_variable("beta1_power", graph=graph), grad.dtype.base_dtype)
          beta2_power = math_ops.cast(self._get_non_slot_variable("beta2_power", graph=graph), grad.dtype.base_dtype)
          lr_t = math_ops.cast(self._lr_t, grad.dtype.base_dtype)
          beta1_t = math_ops.cast(self._beta1_t, grad.dtype.base_dtype)
          beta2_t = math_ops.cast(self._beta2_t, grad.dtype.base_dtype)
          epsilon_t = math_ops.cast(self._epsilon_t, grad.dtype.base_dtype)

          step_size = (lr_t * math_ops.sqrt(1 - beta2_power) / (1 - beta1_power))

          # m_t = beta1 * m + (1 - beta1) * g_t
          m = self.get_slot(var, "m")
          m_scaled_g_values = grad * (1 - beta1_t)
          m_t = state_ops.assign(m, beta1_t * m + m_scaled_g_values, use_locking=self._use_locking)

          # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
          v = self.get_slot(var, "v")
          v_scaled_g_values = (grad - m_t) * (grad - m_t) * (1 - beta2_t)
          v_t = state_ops.assign(v, beta2_t * v + v_scaled_g_values + epsilon_t, use_locking=self._use_locking)

          # amsgrad
          vhat = self.get_slot(var, "vhat")
          if self.amsgrad:
              vhat_t = state_ops.assign(vhat, math_ops.maximum(v_t, vhat))
              v_sqrt = math_ops.sqrt(vhat_t)
          else:
              vhat_t = state_ops.assign(vhat, vhat)
              v_sqrt = math_ops.sqrt(v_t)

          # Compute the bounds
          step_size = step_size / (v_sqrt + epsilon_t)
          bounded_lr = m_t * step_size

          var_update = state_ops.assign_sub(var, bounded_lr, use_locking=self._use_locking)

          return control_flow_ops.group(*[var_update, m_t, v_t, vhat_t])

    def _apply_sparse_shared(self, grad, var, indices, scatter_add):
        beta1_power, beta2_power = self._get_beta_accumulators()
        beta1_power = math_ops.cast(beta1_power, var.dtype.base_dtype)
        beta2_power = math_ops.cast(beta2_power, var.dtype.base_dtype)
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        beta1_t = math_ops.cast(self._beta1_t, var.dtype.base_dtype)
        beta2_t = math_ops.cast(self._beta2_t, var.dtype.base_dtype)
        epsilon_t = math_ops.cast(self._epsilon_t, var.dtype.base_dtype)
        lr = (lr_t * math_ops.sqrt(1 - beta2_power) / (1 - beta1_power))
        # m_t = beta1 * m + (1 - beta1) * g_t
        m = self.get_slot(var, "m")
        m_scaled_g_values = grad * (1 - beta1_t)
        m_t = state_ops.assign(m, m * beta1_t, use_locking=self._use_locking)
        with ops.control_dependencies([m_t]):
            m_t = scatter_add(m, indices, m_scaled_g_values)
        # v_t = beta2 * v + (1 - beta2) * (g_t * g_t)
        v = self.get_slot(var, "v")
        v_scaled_g_values = (grad - m_t) * (grad - m_t) * (1 - beta2_t)
        v_t = state_ops.assign(v, v * beta2_t, use_locking=self._use_locking)
        with ops.control_dependencies([v_t]):
            v_t = scatter_add(v, indices, v_scaled_g_values + epsilon_t)

        # amsgrad
        vhat = self.get_slot(var, "vhat")
        if self.amsgrad:
            vhat_t = state_ops.assign(vhat, math_ops.maximum(v_t, vhat))
            v_sqrt = math_ops.sqrt(vhat_t)
        else:
            vhat_t = state_ops.assign(vhat, vhat)
            v_sqrt = math_ops.sqrt(v_t)

        var_update = state_ops.assign_sub(
            var, lr * m_t / (v_sqrt + epsilon_t), use_locking=self._use_locking)
        return control_flow_ops.group(*[var_update, m_t, v_t, vhat_t])

    def _apply_sparse(self, grad, var):
        return self._apply_sparse_shared(
            grad.values,
            var,
            grad.indices,
            lambda x, i, v: state_ops.scatter_add(  # pylint: disable=g-long-lambda
                x,
                i,
                v,
                use_locking=self._use_locking))

    def _resource_scatter_add(self, x, i, v):
        with ops.control_dependencies(
            [resource_variable_ops.resource_scatter_add(x.handle, i, v)]):
            return x.value()

    def _resource_apply_sparse(self, grad, var, indices):
        return self._apply_sparse_shared(grad, var, indices,
                                         self._resource_scatter_add)

    def _finish(self, update_ops, name_scope):
        # Update the power accumulators.
        with ops.control_dependencies(update_ops):
            beta1_power, beta2_power = self._get_beta_accumulators()
            with ops.colocate_with(beta1_power):
                update_beta1 = beta1_power.assign(
                               beta1_power * self._beta1_t, use_locking=self._use_locking)
                update_beta2 = beta2_power.assign(
                                beta2_power * self._beta2_t, use_locking=self._use_locking)
        return control_flow_ops.group(
            *update_ops + [update_beta1, update_beta2], name=name_scope)

In [15]:
experiment = 'customnet'
experiments = {
    'customnet': '0.001 LR, custom Neural Network.'
}

In [16]:
from keras.optimizers import Adadelta, Adagrad, Adam, Adamax, Nadam, RMSprop, SGD

lr = 0.001
optimizers = [
    ('Yogi', Yogi(lr=lr)),
    ('AdaBelief', AdaBeliefOptimizer(learning_rate=lr)),
    ('RAdam', RAdamOptimizer(learning_rate=lr)),
    ('Adadelta', Adadelta(lr=lr)),
    ('Adagrad', Adagrad(lr=lr)),
    ('Adam', Adam(lr=lr)),
    ('Adamax', Adamax(lr=lr)),
    ('Nadam', Nadam(lr=lr)),
    ('RMSprop', RMSprop(lr=lr)),
    ('SGD', SGD(lr=lr, momentum=0.9, nesterov=True))
]

In [17]:
name, optim = optimizers[3]
optim.get_config()

{'lr': 0.0010000000474974513, 'rho': 0.95, 'decay': 0.0, 'epsilon': 1e-07}

Save optimizer configurations

In [18]:
import os
import pandas as pd
for name, optimizer in optimizers:
    out_dir = os.path.join('models', f'{experiment}_{name}')
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    
    optim_path = os.path.join(out_dir, 'optimizer.json')
    try:
        conf = getattr(optimizer, 'get_config', lambda: {})()
    except:
        conf = {}
    obj = pd.Series(conf)
    obj['name'] = name
    obj['learning_rate'] = lr
    obj.to_json(optim_path)
    print(f'Saved {name} config to {optim_path} ✓')

Saved Yogi config to models/customnet_Yogi/optimizer.json ✓
Saved AdaBelief config to models/customnet_AdaBelief/optimizer.json ✓
Saved RAdam config to models/customnet_RAdam/optimizer.json ✓
Saved Adadelta config to models/customnet_Adadelta/optimizer.json ✓
Saved Adagrad config to models/customnet_Adagrad/optimizer.json ✓
Saved Adam config to models/customnet_Adam/optimizer.json ✓
Saved Adamax config to models/customnet_Adamax/optimizer.json ✓
Saved Nadam config to models/customnet_Nadam/optimizer.json ✓
Saved RMSprop config to models/customnet_RMSprop/optimizer.json ✓
Saved SGD config to models/customnet_SGD/optimizer.json ✓


## Training
Train model, once on every optimizer. Save model once trained; without the optimizer, however. To load the model again, we will have to again compile the model using the optimizer.

In [19]:
for name, optimizer in optimizers:
    print(f'Executing {name}...')
    out_dir = os.path.join('models', f'{experiment}_{name}')
    loss_path = os.path.join(out_dir, 'epoch_loss.csv')
    model_path = os.path.join(out_dir, 'model.h5')
    try:
        my_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])
        my_model.fit_generator(train_generator, epochs=10, validation_data=val_generator, callbacks=[EpochLoss(loss_path)])
        my_model.save(model_path, include_optimizer=False)
    except Exception as e:
        print(optimizer, 'failed', e)
print('All models ran ✨')

Executing Yogi...


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
   3/1224 [..............................] - ETA: 1:58:06 - loss: 6.7605 - acc: 0.0729

KeyboardInterrupt: 

## Save full model

In [20]:
opt = Adam(lr=lr)

In [28]:
my_model = tf.keras.models.load_model('../models/kaggle_Adam/model.h5')
my_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
my_model.save('../demo/public/adam.h5')